# Tutorial on How to Programm Quantum Computers

Author: Tobias Stollenwerk, Institute for Quantum Computing Analytics (PGI-12), FZ Jülich



In [1]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit.visualization import array_to_latex

Load latex packages and define latex macros. See [here](https://docs.mathjax.org/en/latest/input/tex/extensions.html#loading-tex-extensions)

$$
\newcommand{\bra}[1]{\langle #1 |}
\newcommand{\ket}[1]{| #1 \rangle}
\newcommand{\braket}[2]{\langle #1 | #2 \rangle}
\newcommand{\ee}{\mathrm{e}}
\newcommand{\ii}{\mathrm{i}}
$$

## What we will learn

 - Theory
   - What is quantum information
   - How to manipulate quantum information
   - How to build quantum algorithims
 
 - Hands on
   - Construct quantum circuits
   - Use exact simulator to inspect result of gates on quantum states
   - Simulate experiment on real quantum computer and evaluate results
   - Build our first algorithm

 

## Quantum Information - The Qubit

Classical information is stored in bits that can be $0$ or $1$

$$
\text{bit state} \in \{0, 1\}
$$

Quantum information is stored in quantum bits (qubits) that are vectors in a 2-dimensional Hilbert space
$$
\text{qubit state:} \qquad  \ket{\psi} = a \ket{0} + b \ket{1} \in \mathbb{C}^2
$$
with $a,b\in\mathbb{C}$ and $|a|^2 + |b|^2 = 1$.

## Qubit State

 - Representation as vector in $\mathbb{C}^2$: $\ket{\psi} \sim 
 \begin{pmatrix}
a\\
b 
\end{pmatrix}$

- Representation as point on the *Bloch sphere*

  ![image](./images/bloch.png)
  
  $\ket{\psi} = \sin(\theta/2) \ket{0} + \ee^{\ii \phi} \cos(\theta/2) \ket{1}$, where $0\leq\theta\leq\pi, 0\leq\phi\leq 2\pi$ 

## Two-Qubit State

- Consider two distinct qubit states 
$\ket{\psi_1} = a \ket{0} + b \ket{1}$
and
$\ket{\psi_2} = c \ket{0} + d \ket{1}$

- Combined state (*product state*) is described by tensor product

$$
\ket{\psi} = \ket{\psi_1} \otimes \ket{\psi_2} 
= ac \ket{0} \otimes \ket{0} 
+ ad \ket{0} \otimes \ket{1} 
+ bc \ket{1} \otimes \ket{1} 
+ bd \ket{1} \otimes \ket{1}\\
= ac \ket{00} 
+ ad \ket{01} 
+ bc \ket{11}
+ bd \ket{11}
$$

 - In general
 
 $$
\ket{\psi} =
c_{00} \ket{00} 
+ c_{01} \ket{01} 
+ c_{10} \ket{10}
+ c_{11} \ket{11}
$$

- Written in matrix representation

$$ 
\ket{\psi} \sim 
\begin{pmatrix}
c_{00} \\
c_{01} \\ 
c_{10} \\ 
c_{11}
\end{pmatrix}
$$

## $N$-Qubit State

 - In general
 
 $$
\ket{\psi} = \sum_{b \in \{0, 1\}^N} c_{b} \ket{b}
$$

- Written in matrix representation

$$ 
\ket{\psi} \sim 
\begin{pmatrix}
c_{0\dots0} \\
c_{0\dots1} \\
\vdots \\
c_{1\dots0} \\ 
c_{1\dots1}
\end{pmatrix}
\in \mathbb{C}^{2^N}
$$

## How to Manipulate Quantum States - Unitary Transformations

- Transforming a $n$-qubit state $\ket{\psi}$ with the linear transformation $U$
$$
\ket{\psi'} = U \ket{\psi}
$$
- Due to normalization $1 = \braket{\psi}{\psi} = \bra{\psi} U^\dagger U \ket{\psi}$, we have

$$
U^\dagger U = \mathbb{1}
$$

i.e. $U$ is unitary




## Single-Qubit Gate

- General 2x2 matrix
$$
U=
\begin{pmatrix}
u_{00} &
u_{01} \\
u_{10} &
u_{00} 
\end{pmatrix}
$$
- Unitary 2x2 matrix
$$
U=
R_Z(\alpha)
R_Y(\theta)
R_Z(\beta)
=
\begin{pmatrix}
\ee^{\ii \alpha/2} &
0 \\
0 &
\ee^{-\ii \alpha/2}
\end{pmatrix}
\begin{pmatrix}
\cos^{\theta/2} &
\sin^{\theta/2} \\
-\sin^{\theta/2} &
\cos^{\theta/2}
\end{pmatrix}
\begin{pmatrix}
\ee^{\ii \beta/2} &
0 \\
0 &
\ee^{-\ii \beta/2}
\end{pmatrix}
$$

## Important single qubit gates

- Pauli operators
$$
X=
\begin{pmatrix}
0 &
1 \\
1 &
0
\end{pmatrix}
\qquad
Y=
\begin{pmatrix}
0 &
-\ii \\
\ii &
0
\end{pmatrix}
\qquad
Z=
\begin{pmatrix}
1 &
0 \\
0 &
-1
\end{pmatrix}
$$
- Hadamard
$$
H=\frac{1}{\sqrt{2}}
\begin{pmatrix}
1 &
1 \\
-1 &
1
\end{pmatrix}
$$

- Phase- and T-gate
$$
S=
\begin{pmatrix}
1 &
0 \\
-\ii &
0
\end{pmatrix}
\qquad
T=
\begin{pmatrix}
1 &
0 \\
0 &
\ee^{2 \pi \ii/8} 
\end{pmatrix}
$$



## $N$-Qubit Gates

 - Unitary matrix of size $2^N \times 2^N$
 - Example: Two qubit-gate
 
$$
 CX=
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$



## Quantum Circuits

- Multiple choices for basis of $U$
- Example: $\{H, S, T, CX\}$
- Quantum Circuit: Sequential and parallell application of elementary (basis) gates onto an initial state


## Quantum Measurements

- For our purposes, the quantum measurement is a projection onto the some basis states
- Usually we choose the so called compuational basis $\ket{b_0 b_1 \dots b_N}$ where $b\in \{0, 1\}$.
- The probability to measure $\ket{b_0 b_1 \dots b_N}$ given a general $N$-qubit state
$$
\ket{\psi} = \sum_{b \in \text{all bitstrings of length } N} c_b \ket{b}
$$
is $|c_b|^2$.
- For example the probability to measure $0$ in the first and $1$ in the second qubit of the composite state
$$
\ket{\psi} = \frac{1}{\sqrt{2}} (\ket{00} + \ket{11})
$$
is $|\frac{1}{\sqrt{2}}|^2 = 1/2$


## Our first Algorithm: Quantum Teleportation
- We start with Alice (2 qubits, 2 bits) and Bob (1 qubit, 1 bit)
- The first qubit of Alice in is in an unknow quantum state $\ket{\psi}$, she whats to teleport to Bob

![image](images/teleportation01.png)

## Quantum Teleportation - Shared Entangled State

Alice and Bob share an entangled state (Bell state) $\frac{1}{\sqrt{2}} (\ket{00} + \ket{11})$

![image](images/teleportation02.png)

## Quantum Teleportation - Entangling an Measuring

 - Alice entangles both of her qubit (and as a result also Bob's qubit)
 - Then she measures both of her qubits and sends the result (0 or 1) via a classical channel to Bob

 
![image](images/teleportation03.png)

## Quantum Teleportation - Sending the Measurement Outcome and Bob preparing the state

Depending on the measurement results of Alice, Bob applies X (only if Alices measured 1 in the 2nd qubit) and then Z (only if Alice measured 1 in the 1st qubit).
![image](images/teleportation04.png)